In [1]:
# ═══════════════════════════════════════════════════════════════════
# DETECTOR AUTOMÁTICO v2.0 - MELHORADO E INTERATIVO
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import xlrd
import re
import json
import os
import subprocess
import platform
from pathlib import Path
from datetime import datetime
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# GUI
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

print("╔" + "="*78 + "╗")
print("║" + " 🔍 DETECTOR AUTOMÁTICO v2.0 - INTERATIVO".center(78) + "║")
print("╠" + "="*78 + "╣")
print("║" + " Seleção de Sheet | Preview Visual | Confirmação".center(78) + "║")
print("╚" + "="*78 + "╝")
print("\n✅ Imports carregados")

╔==============================================================================╗
║                    🔍 DETECTOR AUTOMÁTICO v2.0 - INTERATIVO                   ║
╠==============================================================================╣
║                Seleção de Sheet | Preview Visual | Confirmação               ║
╚==============================================================================╝

✅ Imports carregados


In [2]:
# ═══════════════════════════════════════════════════════════════════
# FILEMANAGER COM ABERTURA AUTOMÁTICA
# ═══════════════════════════════════════════════════════════════════

class FileManagerInterativo:
    def __init__(self, base_path=None):
        self.base_path = Path(base_path) if base_path else Path.cwd()

        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios'
        }

        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        print(f"✅ FileManager inicializado")
        print(f"   📂 Base: {self.base_path}")
        print(f"   🕐 Timestamp: {self.timestamp}")

    def salvar(self, df, nome, tipo='xlsx', pasta='processados'):
        arquivo = self.pastas[pasta] / f"{nome}_{self.timestamp}.{tipo}"

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        print(f"   💾 {arquivo.name}")
        return arquivo

    def abrir_pasta(self, pasta):
        """Abre pasta no explorador de arquivos"""
        caminho = self.pastas[pasta]

        sistema = platform.system()
        try:
            if sistema == 'Windows':
                os.startfile(caminho)
            elif sistema == 'Darwin':  # macOS
                subprocess.run(['open', caminho])
            else:  # Linux
                subprocess.run(['xdg-open', caminho])
            print(f"\n📂 Pasta aberta: {caminho}")
        except Exception as e:
            print(f"\n⚠️  Não foi possível abrir pasta: {e}")
            print(f"   📂 Caminho: {caminho}")

fm = FileManagerInterativo()

✅ FileManager inicializado
   📂 Base: C:\Users\fpsou\PycharmProjects\AIVI-RECALCULOBatentesLimites
   🕐 Timestamp: 20251015_154712


In [3]:
# ═══════════════════════════════════════════════════════════════════
# DICIONÁRIO DE DADOS PERSISTENTE
# ═══════════════════════════════════════════════════════════════════

class DicionarioDados:
    def __init__(self, fm):
        self.fm = fm
        self.arquivo_dict = fm.pastas['dicionarios'] / 'DICIONARIO_MASTER.json'
        self.padroes = self._carregar_ou_criar()

    def _carregar_ou_criar(self):
        """Carrega dicionário existente ou cria novo"""
        if self.arquivo_dict.exists():
            with open(self.arquivo_dict, 'r', encoding='utf-8') as f:
                dados = json.load(f)
            print(f"✅ Dicionário carregado: {len(dados['padroes_regex'])} padrões")
            return dados
        else:
            print(f"📝 Criando novo dicionário...")
            dados = self._criar_dicionario_padrao()
            self._salvar(dados)
            return dados

    def _criar_dicionario_padrao(self):
        """Dicionário padrão com padrões regex"""
        return {
            'versao': '2.0',
            'ultima_atualizacao': datetime.now().isoformat(),
            'padroes_regex': {
                'Centro': {
                    'regex': r'^[5-9]\d{3}$',
                    'sinonimos': ['Centro', 'Código de Centro', 'Cod Centro', 'Centro Operacional'],
                    'exemplos': ['5025', '5065', '5174']
                },
                'Codigo_Produto': {
                    'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$',
                    'sinonimos': ['Cód Grupo de produto', 'Código Produto', 'Cod Produto'],
                    'exemplos': ['10.123.456', '1234567']
                },
                'Codigo_Grupo': {
                    'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$|^[A-Z_]+$',
                    'sinonimos': ['Cód Grupo de produto', 'Grupo Produto'],
                    'exemplos': ['10.123.456', 'DIESEL_S10_SIMPLES']
                },
                'Doc_Transporte': {
                    'regex': r'^\d{10}$',
                    'sinonimos': ['Documento de transporte', 'Programação'],
                    'exemplos': ['1234567890']
                },
                'Sigla_Base': {
                    'regex': r'^[A-Z]{4,10}$',
                    'sinonimos': ['Sigla', 'Sigla Base', 'Base'],
                    'exemplos': ['BABET', 'BAPLAN', 'AIBET']
                },
                'Percentual': {
                    'regex': r'^-?\d+([.,]\d+)?%?$',
                    'sinonimos': ['%', 'Percentual', 'Perc'],
                    'exemplos': ['10.5%', '-5.2']
                },
                'Monetario': {
                    'regex': r'^R\$\s?-?\d{1,3}(\.\d{3})*(,\d{2})?$|^-?\d+([.,]\d{2})?$',
                    'sinonimos': ['Valor', 'Custo', 'Preço', 'R$'],
                    'exemplos': ['R$ 1.234,56', '1234.56']
                },
                'Numero_Inteiro': {
                    'regex': r'^-?\d+$',
                    'sinonimos': ['Quantidade', 'Qtd', 'Total'],
                    'exemplos': ['123', '-456']
                },
                'Numero_Decimal': {
                    'regex': r'^-?\d+([.,]\d+)?$',
                    'sinonimos': ['Volume', 'Medida'],
                    'exemplos': ['123.45', '123,45']
                },
                'Data_ISO': {
                    'regex': r'^\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])$',
                    'sinonimos': ['Data', 'Período'],
                    'exemplos': ['2024-01-15']
                },
                'Data_BR': {
                    'regex': r'^(0[1-9]|[12]\d|3[01])/(0[1-9]|1[0-2])/\d{4}$',
                    'sinonimos': ['Data'],
                    'exemplos': ['15/01/2024']
                }
            },
            'arquivos_processados': []
        }

    def _salvar(self, dados):
        """Salva dicionário"""
        dados['ultima_atualizacao'] = datetime.now().isoformat()
        with open(self.arquivo_dict, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=2, ensure_ascii=False)

    def detectar_tipo(self, coluna_nome, valores_amostra):
        """Detecta tipo do campo por conteúdo + nome"""
        valores_str = [str(v).strip() for v in valores_amostra if pd.notna(v)]

        if not valores_str:
            return {'tipo': 'VAZIO', 'confianca': 0.0, 'matches': 0, 'total': 0}

        resultados = []

        for nome_tipo, info in self.padroes['padroes_regex'].items():
            # Score por conteúdo
            matches = sum(1 for v in valores_str if re.match(info['regex'], v))
            score_conteudo = matches / len(valores_str)

            # Bonus se nome da coluna está nos sinônimos
            bonus_nome = 0.0
            for sinonimo in info['sinonimos']:
                if sinonimo.lower() in coluna_nome.lower():
                    bonus_nome = 0.2
                    break

            score_final = min(score_conteudo + bonus_nome, 1.0)

            resultados.append({
                'tipo': nome_tipo,
                'confianca': score_final,
                'matches': matches,
                'total': len(valores_str),
                'bonus_nome': bonus_nome > 0
            })

        melhor = max(resultados, key=lambda x: x['confianca'])

        if melhor['confianca'] < 0.70:
            melhor['tipo'] = 'TEXTO_GENERICO'

        return melhor

    def adicionar_arquivo_processado(self, info_arquivo):
        """Adiciona arquivo ao histórico"""
        self.padroes['arquivos_processados'].append(info_arquivo)
        self._salvar(self.padroes)

# Inicializar
dicionario = DicionarioDados(fm)

📝 Criando novo dicionário...


In [4]:
# ═══════════════════════════════════════════════════════════════════
# SELEÇÃO DE ARQUIVO
# ═══════════════════════════════════════════════════════════════════

def selecionar_arquivo_gui():
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)

    arquivo = filedialog.askopenfilename(
        title="Selecione o arquivo Excel",
        initialdir=fm.pastas['entrada'],
        filetypes=[
            ("Arquivos Excel", "*.xlsx *.xls *.xlsm"),
            ("Todos os arquivos", "*.*")
        ]
    )

    root.destroy()

    if not arquivo:
        raise ValueError("❌ Nenhum arquivo selecionado")

    return Path(arquivo)

print("\n" + "="*80)
print("📂 SELEÇÃO DE ARQUIVO")
print("="*80)
print("Abrindo janela... (pode estar atrás do navegador)")

arquivo_selecionado = selecionar_arquivo_gui()

print(f"\n✅ Arquivo selecionado:")
print(f"   📄 Nome: {arquivo_selecionado.name}")
print(f"   📏 Tamanho: {arquivo_selecionado.stat().st_size / 1024:.1f} KB")
print(f"   📂 Local: {arquivo_selecionado.parent}")


📂 SELEÇÃO DE ARQUIVO
Abrindo janela... (pode estar atrás do navegador)

✅ Arquivo selecionado:
   📄 Nome: 2025-2024-YSMM_VI_ACOMP.xlsx
   📏 Tamanho: 3738.1 KB
   📂 Local: F:\Downloads\AIVI


In [5]:
# ═══════════════════════════════════════════════════════════════════
# CARREGAMENTO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📥 CARREGAMENTO DO ARQUIVO")
print("="*80)

try:
    workbook = xlrd.open_workbook(str(arquivo_selecionado))
    sheets = workbook.sheet_names()
    metodo_carga = 'xlrd'
    print(f"✅ Método: xlrd (XLS)")
except:
    workbook = pd.ExcelFile(str(arquivo_selecionado))
    sheets = workbook.sheet_names
    metodo_carga = 'pandas'
    print(f"✅ Método: pandas (XLSX/XLSM)")

print(f"\n📊 Sheets encontradas: {len(sheets)}")
for i, sheet in enumerate(sheets, 1):
    print(f"   {i}. {sheet}")


📥 CARREGAMENTO DO ARQUIVO
✅ Método: pandas (XLSX/XLSM)

📊 Sheets encontradas: 1
   1. 2025-2024-YSMMVIMONITOR


In [6]:
# ═══════════════════════════════════════════════════════════════════
# SELEÇÃO DE SHEET (GUI)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📋 SELEÇÃO DE SHEET")
print("="*80)

def selecionar_sheet_gui(sheets_disponiveis):
    """GUI para seleção de sheet"""
    resultado = {'sheet': None, 'cancelado': False}

    root = tk.Tk()
    root.title("DETECTOR AUTOMÁTICO - Seleção de Sheet")
    root.geometry("600x400")
    root.resizable(False, False)

    # Centralizar
    x = (root.winfo_screenwidth() // 2) - 300
    y = (root.winfo_screenheight() // 2) - 200
    root.geometry(f"+{x}+{y}")
    root.attributes('-topmost', True)

    # Frame principal
    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(fill=tk.BOTH, expand=True)

    # Título
    tk.Label(
        frame,
        text="Selecione a Sheet para processar:",
        font=('Arial', 12, 'bold')
    ).pack(pady=(0, 15))

    # Listbox com scrollbar
    frame_list = tk.Frame(frame)
    frame_list.pack(fill=tk.BOTH, expand=True, pady=(0, 20))

    scrollbar = tk.Scrollbar(frame_list)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    listbox = tk.Listbox(
        frame_list,
        yscrollcommand=scrollbar.set,
        font=('Arial', 10),
        height=10
    )
    listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    scrollbar.config(command=listbox.yview)

    # Adicionar sheets
    for sheet in sheets_disponiveis:
        listbox.insert(tk.END, sheet)

    listbox.select_set(0)  # Selecionar primeira

    # Info
    tk.Label(
        frame,
        text="💡 Clique 2x na sheet OU selecione e clique 'Confirmar'",
        font=('Arial', 9, 'italic'),
        fg='#666666'
    ).pack(pady=(0, 10))

    # Botões
    frame_btns = tk.Frame(frame)
    frame_btns.pack(side=tk.BOTTOM)

    def confirmar():
        selecao = listbox.curselection()
        if selecao:
            resultado['sheet'] = sheets_disponiveis[selecao[0]]
            root.quit()
            root.destroy()
        else:
            messagebox.showwarning("Aviso", "Selecione uma sheet!")

    def cancelar():
        resultado['cancelado'] = True
        root.quit()
        root.destroy()

    def duplo_clique(event):
        confirmar()

    listbox.bind('<Double-Button-1>', duplo_clique)

    tk.Button(
        frame_btns,
        text="Confirmar",
        command=confirmar,
        width=15,
        height=2,
        bg='#4CAF50',
        fg='white',
        font=('Arial', 10, 'bold'),
        cursor='hand2'
    ).pack(side=tk.LEFT, padx=5)

    tk.Button(
        frame_btns,
        text="Cancelar",
        command=cancelar,
        width=15,
        height=2,
        bg='#757575',
        fg='white',
        font=('Arial', 10),
        cursor='hand2'
    ).pack(side=tk.LEFT, padx=5)

    root.mainloop()

    if resultado['cancelado']:
        raise ValueError("❌ Seleção cancelada pelo usuário")

    return resultado['sheet']

# Executar seleção
print("Abrindo janela de seleção...")
sheet_nome = selecionar_sheet_gui(sheets)

print(f"\n✅ Sheet selecionada: '{sheet_nome}'")


📋 SELEÇÃO DE SHEET
Abrindo janela de seleção...

✅ Sheet selecionada: '2025-2024-YSMMVIMONITOR'


In [7]:
# ═══════════════════════════════════════════════════════════════════
# PREVIEW VISUAL (50 linhas × 15 colunas)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("👀 PREVIEW DO ARQUIVO")
print("="*80)

# Carregar preview
if metodo_carga == 'xlrd':
    sheet = workbook.sheet_by_name(sheet_nome)
    data_preview = [sheet.row_values(i) for i in range(min(50, sheet.nrows))]
    df_preview = pd.DataFrame(data_preview)
else:
    df_preview = pd.read_excel(workbook, sheet_name=sheet_nome, nrows=50, header=None)

# Limitar a 15 colunas
df_preview_limitado = df_preview.iloc[:, :15].copy()

print(f"\n📊 Mostrando primeiras 50 linhas × 15 colunas")
print(f"   (Total no arquivo: {len(df_preview)} linhas × {len(df_preview.columns)} colunas)")

# Adicionar coluna de índice EXCEL (começa em 1)
df_preview_exibir = df_preview_limitado.copy()
df_preview_exibir.insert(0, 'LINHA EXCEL', range(1, len(df_preview_exibir) + 1))

print("\n" + "─"*80)
display(df_preview_exibir)
print("─"*80)

print("\n💡 LEGENDA:")
print("   • Coluna 'LINHA EXCEL' = número da linha no Excel")
print("   • Índice à esquerda (0, 1, 2...) = índice Python")
print("   • Identifique qual linha contém os NOMES DAS COLUNAS")


👀 PREVIEW DO ARQUIVO

📊 Mostrando primeiras 50 linhas × 15 colunas
   (Total no arquivo: 50 linhas × 30 colunas)

────────────────────────────────────────────────────────────────────────────────


,LINHA EXCEL,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,08.10.2025 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,Nome do set,Mês do exercício,Ano do documento do material,Centro,Cód Grupo de produto,Desc. Grupo de Produto,Nome,Expedição c/ Veí,Variação Interna,Variação Manual,VarInt + VarMan,Percentual de V,Limite Inferior,Limite Su
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,NaN,NaN,10,2025,5174,ETANOL_ADITIVADO,ETANOL ADITIVADO,BADEN Base de PresidPrudente,150837,0,0,0,0,0,0
6,7,NaN,NaN,10,2025,5174,GASOLINA_ADITIVADA,GASOLINA ADITIVADA,BADEN Base de PresidPrudente,222017,0,0,0,0,0,0
7,8,NaN,NaN,10,2025,5174,GASOLINA_COMPOSTO,GASOLINA,BADEN Base de PresidPrudente,1159915,0,0,0,0,0,0
8,9,NaN,NaN,10,2025,5174,GASOLINA_SIMPLES,GASOLINA,BADEN Base de PresidPrudente,1209443,-4659.5,0,-4659.5,-0.39,0,0
9,10,NaN,NaN,10,2025,5174,HIDRATADO_SIMPLES,HIDRATADO,BADEN Base de PresidPrudente,1703289,919,0,919,0.05,0,0


────────────────────────────────────────────────────────────────────────────────

💡 LEGENDA:
   • Coluna 'LINHA EXCEL' = número da linha no Excel
   • Índice à esquerda (0, 1, 2...) = índice Python
   • Identifique qual linha contém os NOMES DAS COLUNAS


In [8]:
# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO E CONFIRMAÇÃO DE CABEÇALHO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🎯 DETECÇÃO DE CABEÇALHO")
print("="*80)

def avaliar_linha_cabecalho(linha, idx, dicionario):
    """Score para candidato a cabeçalho"""
    celulas = [str(c).strip() for c in linha if str(c).strip()]

    if not celulas:
        return 0.0

    score = 0.0

    # Proporção não vazias
    score += (len(celulas) / len(linha)) * 2.0

    # Tem texto
    tem_texto = sum(1 for c in celulas if re.search(r'[a-zA-Z]', c))
    score += (tem_texto / len(celulas)) * 1.5

    # Match com dicionário (NOVO)
    bonus_dicionario = 0.0
    for celula in celulas:
        for tipo, info in dicionario.padroes['padroes_regex'].items():
            for sinonimo in info['sinonimos']:
                if sinonimo.lower() in celula.lower():
                    bonus_dicionario += 0.5
                    break
    score += min(bonus_dicionario, 3.0)

    # Tamanho médio
    tamanho_medio = np.mean([len(c) for c in celulas])
    if 5 <= tamanho_medio <= 50:
        score += 1.0

    # Unicidade
    if len(celulas) == len(set(celulas)):
        score += 1.5

    # Posição
    if idx < 50:
        score += (50 - idx) / 100

    return score

# Avaliar linhas
scores = []
for idx, linha in enumerate(data_preview if metodo_carga == 'xlrd' else df_preview.values.tolist()):
    score = avaliar_linha_cabecalho(linha, idx, dicionario)
    scores.append({
        'linha_excel': idx + 1,
        'indice_python': idx,
        'score': score,
        'conteudo': linha[:5]
    })

scores = sorted(scores, key=lambda x: x['score'], reverse=True)

print("\n🏆 Top 5 candidatos:")
for i, item in enumerate(scores[:5], 1):
    print(f"   {i}. Linha {item['linha_excel']} (Excel) = Índice {item['indice_python']} (Python) - Score: {item['score']:.2f}")
    print(f"      Conteúdo: {item['conteudo'][:3]}")

melhor_candidato = scores[0]

print(f"\n🎯 DETECÇÃO AUTOMÁTICA:")
print(f"   Linha Excel: {melhor_candidato['linha_excel']}")
print(f"   Índice Python: {melhor_candidato['indice_python']}")
print(f"   Confiança: {melhor_candidato['score']:.2f}/10")

# CONFIRMAÇÃO INTERATIVA
def confirmar_cabecalho_gui(candidato_sugerido, max_linhas):
    """GUI para confirmar ou corrigir cabeçalho"""
    resultado = {'linha_excel': None, 'cancelado': False}

    root = tk.Tk()
    root.title("DETECTOR AUTOMÁTICO - Confirmar Cabeçalho")
    root.geometry("500x300")
    root.resizable(False, False)

    x = (root.winfo_screenwidth() // 2) - 250
    y = (root.winfo_screenheight() // 2) - 150
    root.geometry(f"+{x}+{y}")
    root.attributes('-topmost', True)

    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(fill=tk.BOTH, expand=True)

    # Mensagem
    tk.Label(
        frame,
        text="Confirme a linha do CABEÇALHO:",
        font=('Arial', 12, 'bold')
    ).pack(pady=(0, 10))

    tk.Label(
        frame,
        text=f"🤖 Detecção automática sugere: Linha {candidato_sugerido}",
        font=('Arial', 10),
        fg='#2196F3'
    ).pack(pady=(0, 5))

    tk.Label(
        frame,
        text="(Linha que contém os NOMES das colunas)",
        font=('Arial', 9, 'italic'),
        fg='#666666'
    ).pack(pady=(0, 20))

    # Frame de input
    frame_input = tk.Frame(frame)
    frame_input.pack(pady=(0, 20))

    tk.Label(
        frame_input,
        text="Número da linha (Excel):",
        font=('Arial', 10)
    ).pack(side=tk.LEFT, padx=(0, 10))

    var_linha = tk.StringVar(value=str(candidato_sugerido))
    entry = tk.Entry(
        frame_input,
        textvariable=var_linha,
        font=('Arial', 12),
        width=10
    )
    entry.pack(side=tk.LEFT)
    entry.select_range(0, tk.END)
    entry.focus()

    # Validação
    label_aviso = tk.Label(
        frame,
        text="",
        font=('Arial', 9),
        fg='#FF0000'
    )
    label_aviso.pack(pady=(0, 10))

    def validar_e_confirmar():
        try:
            linha_excel = int(var_linha.get())
            if 1 <= linha_excel <= max_linhas:
                resultado['linha_excel'] = linha_excel
                root.quit()
                root.destroy()
            else:
                label_aviso.config(text=f"❌ Linha deve estar entre 1 e {max_linhas}")
        except ValueError:
            label_aviso.config(text="❌ Digite um número válido")

    def cancelar():
        resultado['cancelado'] = True
        root.quit()
        root.destroy()

    def enter_pressed(event):
        validar_e_confirmar()

    entry.bind('<Return>', enter_pressed)

    # Botões
    frame_btns = tk.Frame(frame)
    frame_btns.pack(side=tk.BOTTOM)

    tk.Button(
        frame_btns,
        text="Confirmar",
        command=validar_e_confirmar,
        width=12,
        height=2,
        bg='#4CAF50',
        fg='white',
        font=('Arial', 10, 'bold'),
        cursor='hand2'
    ).pack(side=tk.LEFT, padx=5)

    tk.Button(
        frame_btns,
        text="Cancelar",
        command=cancelar,
        width=12,
        height=2,
        bg='#757575',
        fg='white',
        font=('Arial', 10),
        cursor='hand2'
    ).pack(side=tk.LEFT, padx=5)

    root.mainloop()

    if resultado['cancelado']:
        raise ValueError("❌ Confirmação cancelada")

    return resultado['linha_excel']

# Executar confirmação
print("\n📋 Abrindo janela de confirmação...")
linha_cabecalho_excel = confirmar_cabecalho_gui(
    melhor_candidato['linha_excel'],
    len(df_preview)
)

# Converter para índice Python
linha_cabecalho = linha_cabecalho_excel - 1
linha_dados_inicio = linha_cabecalho + 1

print(f"\n✅ Cabeçalho confirmado:")
print(f"   📍 Linha Excel: {linha_cabecalho_excel}")
print(f"   📍 Índice Python: {linha_cabecalho}")
print(f"   📊 Dados iniciam na linha Excel: {linha_dados_inicio + 1}")


🎯 DETECÇÃO DE CABEÇALHO

🏆 Top 5 candidatos:
   1. Linha 4 (Excel) = Índice 3 (Python) - Score: 9.47
      Conteúdo: [nan, 'Nome do set', 'Mês do exercício']
   2. Linha 1 (Excel) = Índice 0 (Python) - Score: 5.00
      Conteúdo: ['08.10.2025                                                                                                                                                                                                                                             Saída dinâmica de lista                                                                                                                                                                                                                                                    1', nan, nan]
   3. Linha 9 (Excel) = Índice 8 (Python) - Score: 4.47
      Conteúdo: [nan, nan, 10]
   4. Linha 10 (Excel) = Índice 9 (Python) - Score: 4.46
      Conteúdo: [nan, nan, 10]
   5. Linha 39 (Excel) = Índice 38 (Python) - Score: 4.12
      

In [9]:
# ═══════════════════════════════════════════════════════════════════
# EXTRAÇÃO DE DADOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📥 EXTRAÇÃO DE DADOS")
print("="*80)

if metodo_carga == 'xlrd':
    sheet = workbook.sheet_by_name(sheet_nome)
    data = [sheet.row_values(i) for i in range(sheet.nrows)]
    df_bruto = pd.DataFrame(data)

    cabecalho_bruto = df_bruto.iloc[linha_cabecalho].tolist()
    df_bruto = df_bruto.iloc[linha_dados_inicio:].copy()
    df_bruto.columns = cabecalho_bruto
else:
    df_bruto = pd.read_excel(workbook, sheet_name=sheet_nome, header=linha_cabecalho)

df_bruto = df_bruto.reset_index(drop=True)

print(f"✅ Dados extraídos:")
print(f"   📊 {len(df_bruto):,} registros")
print(f"   📋 {len(df_bruto.columns)} colunas")
print(f"   💾 {df_bruto.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


📥 EXTRAÇÃO DE DADOS
✅ Dados extraídos:
   📊 27,656 registros
   📋 30 colunas
   💾 20.67 MB


In [10]:
# ═══════════════════════════════════════════════════════════════════
# LIMPEZA DE ESTRUTURA
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🧹 LIMPEZA DE ESTRUTURA")
print("="*80)

df = df_bruto.copy()
log_limpeza = []

# 1. Colunas vazias
colunas_vazias = df.columns[df.isna().all()].tolist()
if colunas_vazias:
    print(f"🗑️  Removendo {len(colunas_vazias)} colunas completamente vazias")
    df = df.drop(columns=colunas_vazias)
    log_limpeza.append(f"Removidas {len(colunas_vazias)} colunas vazias")

# 2. Linhas vazias
linhas_vazias = df.isna().all(axis=1).sum()
if linhas_vazias > 0:
    print(f"🗑️  Removendo {linhas_vazias} linhas completamente vazias")
    df = df.dropna(how='all')
    log_limpeza.append(f"Removidas {linhas_vazias} linhas vazias")

# 3. Limpar nomes de colunas
print("🧹 Limpando nomes de colunas...")
colunas_originais = df.columns.tolist()
colunas_limpas = []
for col in df.columns:
    col_limpo = str(col).strip().lstrip("'").replace('\n', ' ').replace('\r', '')
    col_limpo = ' '.join(col_limpo.split())
    colunas_limpas.append(col_limpo)
df.columns = colunas_limpas

colunas_modificadas = sum(1 for orig, limpo in zip(colunas_originais, colunas_limpas) if orig != limpo)
if colunas_modificadas > 0:
    print(f"   ✅ {colunas_modificadas} nomes limpos")
    log_limpeza.append(f"Limpeza de nomes: {colunas_modificadas} colunas")

# 4. Renomear duplicadas
contagem = Counter(colunas_limpas)
duplicadas = {c: n for c, n in contagem.items() if n > 1}

if duplicadas:
    print(f"⚠️  Renomeando {len(duplicadas)} colunas duplicadas:")
    colunas_finais = []
    contador = {}

    for col in colunas_limpas:
        if col in duplicadas:
            if col not in contador:
                contador[col] = 0
                colunas_finais.append(col)
            else:
                contador[col] += 1
                novo = f"{col}_dup{contador[col]}"
                colunas_finais.append(novo)
                print(f"   '{col}' → '{novo}'")
        else:
            colunas_finais.append(col)

    df.columns = colunas_finais
    log_limpeza.append(f"Renomeadas {sum(contador.values())} colunas duplicadas")

# 5. Remover linhas de totais
padroes_remover = [r'(?i)^total', r'(?i)^resultado', r'(?i)^soma', r'(?i)^subtotal']
linhas_remover = []

for idx, row in df.iterrows():
    primeira_celula = str(row.iloc[0]).strip().lower()
    if any(re.search(p, primeira_celula) for p in padroes_remover):
        linhas_remover.append(idx)

if linhas_remover:
    print(f"🗑️  Removendo {len(linhas_remover)} linhas de totais/resultados")
    df = df.drop(index=linhas_remover)
    log_limpeza.append(f"Removidas {len(linhas_remover)} linhas de totais")

df = df.reset_index(drop=True)
df_limpo = df.copy()

print(f"\n✅ Limpeza concluída:")
print(f"   📊 Registros: {len(df_bruto):,} → {len(df_limpo):,}")
print(f"   📋 Colunas: {len(df_bruto.columns)} → {len(df_limpo.columns)}")
print(f"   📝 {len(log_limpeza)} operações realizadas")


🧹 LIMPEZA DE ESTRUTURA
🗑️  Removendo 1 colunas completamente vazias
🗑️  Removendo 1 linhas completamente vazias
🧹 Limpando nomes de colunas...
   ✅ 1 nomes limpos

✅ Limpeza concluída:
   📊 Registros: 27,656 → 27,655
   📋 Colunas: 30 → 29
   📝 3 operações realizadas


In [11]:
# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO DE TIPOS POR CONTEÚDO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🔬 DETECÇÃO DE TIPOS POR CONTEÚDO")
print("="*80)

tipos_detectados = {}

for col in df_limpo.columns:
    valores_amostra = df_limpo[col].dropna().head(100).tolist()
    tipo_info = dicionario.detectar_tipo(col, valores_amostra)
    tipos_detectados[col] = tipo_info

    # Emoji baseado em confiança
    if tipo_info['confianca'] >= 0.90:
        emoji = "✅"
        cor = "ALTA"
    elif tipo_info['confianca'] >= 0.70:
        emoji = "⚠️ "
        cor = "MÉDIA"
    else:
        emoji = "❓"
        cor = "BAIXA"

    bonus_str = " (match nome)" if tipo_info.get('bonus_nome') else ""

    print(f"{emoji} {col[:35]:35s} → {tipo_info['tipo']:20s} | Confiança: {cor:6s} ({tipo_info['confianca']:.0%}){bonus_str}")

print(f"\n✅ Análise concluída: {len(tipos_detectados)} colunas")


🔬 DETECÇÃO DE TIPOS POR CONTEÚDO
✅ Nome do set                         → Codigo_Grupo         | Confiança: ALTA   (100%)
✅ Mês do exercício                    → Percentual           | Confiança: ALTA   (100%)
✅ Ano do documento do material        → Percentual           | Confiança: ALTA   (100%)
✅ Centro                              → Percentual           | Confiança: ALTA   (100%)
⚠️  Cód Grupo de produto                → Codigo_Grupo         | Confiança: MÉDIA  (83%) (match nome)
❓ Desc. Grupo de Produto              → TEXTO_GENERICO       | Confiança: BAIXA  (40%)
❓ Nome                                → TEXTO_GENERICO       | Confiança: BAIXA  (3%)
✅ Expedição c/ Veí                    → Percentual           | Confiança: ALTA   (100%)
✅ Variação Interna                    → Percentual           | Confiança: ALTA   (100%)
✅ Variação Manual                     → Percentual           | Confiança: ALTA   (100%)
✅ VarInt + VarMan                     → Percentual           | Confiança: A

In [12]:
# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÕES E ESTATÍSTICAS VISUAIS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("📊 VALIDAÇÕES E ESTATÍSTICAS")
print("="*80)

# 1. Resumo Geral
print("\n📋 RESUMO GERAL:")
print("─" * 80)
print(f"   Registros finais: {len(df_limpo):,}")
print(f"   Colunas finais: {len(df_limpo.columns)}")
print(f"   Memória em uso: {df_limpo.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   Linhas duplicadas: {df_limpo.duplicated().sum():,}")

# 2. Valores Nulos
print("\n🔍 ANÁLISE DE VALORES NULOS:")
print("─" * 80)
nulos = df_limpo.isna().sum()
colunas_com_nulos = nulos[nulos > 0].sort_values(ascending=False)

if len(colunas_com_nulos) == 0:
    print("   ✅ Nenhuma coluna com valores nulos")
else:
    print(f"   ⚠️  {len(colunas_com_nulos)} colunas com valores nulos:")
    for col, qtd in colunas_com_nulos.head(10).items():
        perc = (qtd / len(df_limpo)) * 100
        barra = "█" * int(perc / 5)  # Barra visual
        print(f"      {col[:30]:30s} | {qtd:>6,} ({perc:>5.1f}%) {barra}")

# 3. Distribuição de Tipos
print("\n🔬 DISTRIBUIÇÃO DE TIPOS DETECTADOS:")
print("─" * 80)
tipos_resumo = {}
for info in tipos_detectados.values():
    tipo = info['tipo']
    tipos_resumo[tipo] = tipos_resumo.get(tipo, 0) + 1

for tipo, count in sorted(tipos_resumo.items(), key=lambda x: x[1], reverse=True):
    barra = "█" * count
    print(f"   {tipo:25s} | {count:>2} colunas {barra}")

# 4. Cardinalidade
print("\n📊 ANÁLISE DE CARDINALIDADE:")
print("─" * 80)
print("   (Colunas com poucos valores únicos podem ser categóricas)\n")

for col in df_limpo.columns:
    n_unicos = df_limpo[col].nunique()
    perc_card = (n_unicos / len(df_limpo)) * 100

    if perc_card < 5:  # Menos de 5% de valores únicos
        emoji = "🏷️ "
        status = "CATEGÓRICA"
    elif perc_card < 50:
        emoji = "📊"
        status = "SEMI-CATEGÓRICA"
    else:
        emoji = "📈"
        status = "ALTA CARDINALIDADE"

    if perc_card <= 50:  # Mostrar apenas categorias e semi
        print(f"   {emoji} {col[:30]:30s} | {n_unicos:>6,} únicos ({perc_card:>5.1f}%) - {status}")

# 5. Visualização de amostra
print("\n👀 AMOSTRA DOS DADOS LIMPOS:")
print("─" * 80)
display(df_limpo.head(10))

print("\n📊 ESTATÍSTICAS DESCRITIVAS:")
print("─" * 80)
display(df_limpo.describe(include='all'))


📊 VALIDAÇÕES E ESTATÍSTICAS

📋 RESUMO GERAL:
────────────────────────────────────────────────────────────────────────────────
   Registros finais: 27,655
   Colunas finais: 29
   Memória em uso: 20.46 MB
   Linhas duplicadas: 0

🔍 ANÁLISE DE VALORES NULOS:
────────────────────────────────────────────────────────────────────────────────
   ⚠️  2 colunas com valores nulos:
      Status                         | 25,807 ( 93.3%) ██████████████████
      Nome do set                    |  1,492 (  5.4%) █

🔬 DISTRIBUIÇÃO DE TIPOS DETECTADOS:
────────────────────────────────────────────────────────────────────────────────
   Percentual                | 20 colunas ████████████████████
   Codigo_Grupo              |  5 colunas █████
   TEXTO_GENERICO            |  4 colunas ████

📊 ANÁLISE DE CARDINALIDADE:
────────────────────────────────────────────────────────────────────────────────
   (Colunas com poucos valores únicos podem ser categóricas)

   🏷️  Nome do set                    |      4

,Nome do set,Mês do exercício,Ano do documento do material,Centro,Cód Grupo de produto,Desc. Grupo de Produto,Nome,Expedição c/ Veí,Variação Interna,Variação Manual,...,Imposto (R$),Valor Exced. da,Valor da VI (R$),Valor da VI +,Perda ou So,Competência,Status de Homologação,Desc Status,Icone,Status
0,NaN,10.0,2025.0,5174.0,ETANOL_ADITIVADO,ETANOL ADITIVADO,BADEN Base de PresidPrudente,150837.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
1,NaN,10.0,2025.0,5174.0,GASOLINA_ADITIVADA,GASOLINA ADITIVADA,BADEN Base de PresidPrudente,222017.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
2,NaN,10.0,2025.0,5174.0,GASOLINA_COMPOSTO,GASOLINA,BADEN Base de PresidPrudente,1159915.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
3,NaN,10.0,2025.0,5174.0,GASOLINA_SIMPLES,GASOLINA,BADEN Base de PresidPrudente,1209443.0,-4659.5,0.0,...,0.0,-27535.3153,-27535.3153,-27535.3153,P,SUPER,A,Aguardando Envio para Estoquistas,@0A@,NaN
4,NaN,10.0,2025.0,5174.0,HIDRATADO_SIMPLES,HIDRATADO,BADEN Base de PresidPrudente,1703289.0,919.0,0.0,...,0.0,2703.9737,2703.9737,2703.9737,S,LM,A,Aguardando Envio para Estoquistas,@0A@,NaN
5,NaN,10.0,2025.0,5179.0,QUEROSENE_SIMPLES,QUEROSENE,RASGA Rev.Avia.S.G.Cachoeira,24335.0,117.0,0.0,...,0.0,646.2729,646.2729,646.2729,S,LM,A,Aguardando Envio para Estoquistas,@0A@,NaN
6,NaN,10.0,2025.0,5182.0,GASOLINA_COMPOSTO,GASOLINA,RAPUC Rev.Avia.Porto Urucu,755.0,-124.0,0.0,...,0.0,-962.0664,-962.0664,-962.0664,P,LM,A,Aguardando Envio para Estoquistas,@0A@,NaN
7,NaN,10.0,2025.0,5182.0,QUEROSENE_SIMPLES,QUEROSENE,RAPUC Rev.Avia.Porto Urucu,31653.0,38.0,0.0,...,0.0,289.1800,289.1800,289.1800,S,LM,A,Aguardando Envio para Estoquistas,@0A@,NaN
8,NaN,10.0,2025.0,5183.0,QUEROSENE_SIMPLES,QUEROSENE,RATEF Rev.Avia.Tefe,17877.0,206.0,0.0,...,0.0,1050.0850,1050.0850,1050.0850,S,LM,A,Aguardando Envio para Estoquistas,@0A@,NaN
9,NaN,10.0,2025.0,5184.0,DIESEL_S10_COMPOSTO,DIESEL S10,ARJUR Arm Conj de Juruti,0.0,387.0,0.0,...,0.0,2075.5971,2075.5971,2075.5971,S,LM,A,Aguardando Envio para Estoquistas,@0A@,NaN



📊 ESTATÍSTICAS DESCRITIVAS:
────────────────────────────────────────────────────────────────────────────────


,Nome do set,Mês do exercício,Ano do documento do material,Centro,Cód Grupo de produto,Desc. Grupo de Produto,Nome,Expedição c/ Veí,Variação Interna,Variação Manual,...,Imposto (R$),Valor Exced. da,Valor da VI (R$),Valor da VI +,Perda ou So,Competência,Status de Homologação,Desc Status,Icone,Status
count,26163,27655.000000,27655.000000,27655.000000,27655,27655,27655,2.765500e+04,27655.000000,2.765500e+04,...,2.765500e+04,2.765500e+04,2.765500e+04,2.765500e+04,27655,27655,27655,27655,27655,1848.000000
unique,4,NaN,NaN,NaN,419,417,182,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,6,3,11,3,NaN
top,OPSUL,NaN,NaN,NaN,DIESEL_S10_COMPOSTO,GASOLINA,BACUB Base de Cubatão,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
freq,9467,NaN,NaN,NaN,1653,3126,956,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,20208,19151,25807,25807,25807,NaN
mean,NaN,6.272428,2024.483276,5252.997758,NaN,NaN,NaN,5.114944e+06,-106.675926,-2.321973e+02,...,3.164352e+02,-5.041009e+02,-1.311480e+03,-9.950451e+02,NaN,NaN,NaN,NaN,NaN,27.661255
std,NaN,3.259079,0.499729,332.127562,NaN,NaN,NaN,1.178486e+07,8101.095234,3.862159e+04,...,8.226597e+04,1.366813e+05,1.206517e+05,1.439200e+05,NaN,NaN,NaN,NaN,NaN,7.588637
min,NaN,1.000000,2024.000000,5012.000000,NaN,NaN,NaN,0.000000e+00,-225714.920000,-4.503640e+06,...,-1.831637e+05,-7.991427e+06,-8.047409e+06,-8.047409e+06,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,3.000000,2024.000000,5095.000000,NaN,NaN,NaN,8.942200e+04,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,-1.490115e+01,-1.417040e+01,NaN,NaN,NaN,NaN,NaN,30.000000
50%,NaN,6.000000,2024.000000,5250.000000,NaN,NaN,NaN,1.125875e+06,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,30.000000
75%,NaN,9.000000,2025.000000,5306.000000,NaN,NaN,NaN,4.646356e+06,5.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,1.065450e+02,1.078089e+02,NaN,NaN,NaN,NaN,NaN,30.000000


In [13]:
# ═══════════════════════════════════════════════════════════════════
# EXPORTAÇÃO COMPLETA COM LOGS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("💾 EXPORTAÇÃO DE RESULTADOS")
print("="*80)

nome_base = arquivo_selecionado.stem

# 1. Dados limpos
print("\n1️⃣  Dados limpos...")
arquivo_limpo = fm.salvar(df_limpo, f"{nome_base}_Limpo", tipo='xlsx', pasta='processados')

# 2. Dicionário de campos
print("\n2️⃣  Dicionário de campos...")
registros_dict = []
for col in df_limpo.columns:
    tipo_info = tipos_detectados.get(col, {})

    # Exemplos de valores
    valores_exemplo = df_limpo[col].dropna().unique()[:3].tolist()

    registros_dict.append({
        'Coluna': col,
        'Tipo_Detectado': tipo_info.get('tipo', 'DESCONHECIDO'),
        'Confianca_%': tipo_info.get('confianca', 0.0) * 100,
        'Match_Nome': 'Sim' if tipo_info.get('bonus_nome') else 'Não',
        'Dtype_Atual': str(df_limpo[col].dtype),
        'Valores_Unicos': df_limpo[col].nunique(),
        'Nulos_Qtd': df_limpo[col].isna().sum(),
        'Nulos_%': (df_limpo[col].isna().sum() / len(df_limpo)) * 100,
        'Exemplo_1': str(valores_exemplo[0]) if len(valores_exemplo) > 0 else None,
        'Exemplo_2': str(valores_exemplo[1]) if len(valores_exemplo) > 1 else None,
        'Exemplo_3': str(valores_exemplo[2]) if len(valores_exemplo) > 2 else None
    })

df_dict = pd.DataFrame(registros_dict)
arquivo_dict = fm.salvar(df_dict, f"DICT_{nome_base}", tipo='xlsx', pasta='outputs')

# 3. Log de processamento (PARA REPRODUÇÃO)
print("\n3️⃣  Log de processamento...")
log_processamento = {
    'Arquivo_Original': arquivo_selecionado.name,
    'Caminho_Original': str(arquivo_selecionado),
    'Sheet_Processada': sheet_nome,
    'Metodo_Carga': metodo_carga,
    'Linha_Cabecalho_Excel': linha_cabecalho_excel,
    'Indice_Cabecalho_Python': linha_cabecalho,
    'Linha_Dados_Inicio_Excel': linha_dados_inicio + 1,
    'Indice_Dados_Inicio_Python': linha_dados_inicio,
    'Registros_Bruto': len(df_bruto),
    'Colunas_Bruto': len(df_bruto.columns),
    'Registros_Limpo': len(df_limpo),
    'Colunas_Limpo': len(df_limpo.columns),
    'Operacoes_Limpeza': ', '.join(log_limpeza),
    'Timestamp': fm.timestamp,
    'Data_Processamento': datetime.now().isoformat()
}

df_log = pd.DataFrame([log_processamento])
arquivo_log = fm.salvar(df_log, f"LOG_{nome_base}", tipo='xlsx', pasta='logs')

# 4. Código Python para reprodução
print("\n4️⃣  Código Python para reprodução...")
codigo_reproducao = f'''# ═══════════════════════════════════════════════════════════════════
# CÓDIGO PARA REPRODUZIR CARREGAMENTO
# Gerado automaticamente pelo Detector Automático v2.0
# Data: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
from pathlib import Path

# Configurações detectadas
arquivo = Path(r"{arquivo_selecionado}")
sheet_nome = "{sheet_nome}"
linha_cabecalho = {linha_cabecalho}  # Índice Python (linha Excel {linha_cabecalho_excel})

# Carregar dados
df = pd.read_excel(
    arquivo,
    sheet_name=sheet_nome,
    header=linha_cabecalho
)

print(f"✅ Carregado: {{len(df):,}} registros × {{len(df.columns)}} colunas")

# Colunas detectadas
colunas_esperadas = {df_limpo.columns.tolist()}

# Validar estrutura
if df.columns.tolist() == colunas_esperadas:
    print("✅ Estrutura validada!")
else:
    print("⚠️  Estrutura diferente do esperado")
    print(f"   Esperado: {{len(colunas_esperadas)}} colunas")
    print(f"   Obtido: {{len(df.columns)}} colunas")
'''

arquivo_codigo = fm.pastas['outputs'] / f"CODIGO_Reproducao_{nome_base}_{fm.timestamp}.py"
with open(arquivo_codigo, 'w', encoding='utf-8') as f:
    f.write(codigo_reproducao)
print(f"   💾 {arquivo_codigo.name}")

# 5. Atualizar dicionário master
print("\n5️⃣  Atualizando dicionário master...")
info_arquivo = {
    'arquivo': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'timestamp': fm.timestamp,
    'colunas': df_limpo.columns.tolist(),
    'registros': len(df_limpo),
    'tipos_detectados': {col: info['tipo'] for col, info in tipos_detectados.items()}
}
dicionario.adicionar_arquivo_processado(info_arquivo)
print(f"   ✅ Dicionário master atualizado")

print("\n✅ EXPORTAÇÃO CONCLUÍDA!")


💾 EXPORTAÇÃO DE RESULTADOS

1️⃣  Dados limpos...
   💾 2025-2024-YSMM_VI_ACOMP_Limpo_20251015_154712.xlsx

2️⃣  Dicionário de campos...
   💾 DICT_2025-2024-YSMM_VI_ACOMP_20251015_154712.xlsx

3️⃣  Log de processamento...
   💾 LOG_2025-2024-YSMM_VI_ACOMP_20251015_154712.xlsx

4️⃣  Código Python para reprodução...
   💾 CODIGO_Reproducao_2025-2024-YSMM_VI_ACOMP_20251015_154712.py

5️⃣  Atualizando dicionário master...
   ✅ Dicionário master atualizado

✅ EXPORTAÇÃO CONCLUÍDA!


In [14]:
# ═══════════════════════════════════════════════════════════════════
# RELATÓRIO FINAL DESTACADO
# ═══════════════════════════════════════════════════════════════════

print("\n\n")
print("╔" + "="*78 + "╗")
print("║" + " 📋 RELATÓRIO FINAL - PROCESSAMENTO CONCLUÍDO".center(78) + "║")
print("╚" + "="*78 + "╝")

print("\n" + "┏" + "━"*78 + "┓")
print("┃" + " 📁 INFORMAÇÕES DO ARQUIVO".center(78) + "┃")
print("┣" + "━"*78 + "┫")
print(f"┃  Nome: {arquivo_selecionado.name:<68}┃")
print(f"┃  Sheet: {sheet_nome:<67}┃")
print(f"┃  Cabeçalho: Linha {linha_cabecalho_excel} (Excel) / Índice {linha_cabecalho} (Python){' '*20}┃")
print(f"┃  Dados: A partir da linha {linha_dados_inicio + 1} (Excel){' '*37}┃")
print("┗" + "━"*78 + "┛")

print("\n" + "┏" + "━"*78 + "┓")
print("┃" + " 📊 RESULTADO DO PROCESSAMENTO".center(78) + "┃")
print("┣" + "━"*78 + "┫")
print(f"┃  📈 Registros: {len(df_bruto):>6,} (original) → {len(df_limpo):>6,} (limpo){' '*28}┃")
print(f"┃  📋 Colunas:   {len(df_bruto.columns):>6} (original) → {len(df_limpo.columns):>6} (limpo){' '*28}┃")
print(f"┃  💾 Memória:   {df_limpo.memory_usage(deep=True).sum() / 1024**2:>6.2f} MB{' '*50}┃")
print(f"┃  🔬 Tipos detectados: {len(set(info['tipo'] for info in tipos_detectados.values())):>2} tipos únicos{' '*32}┃")
print("┗" + "━"*78 + "┛")

print("\n" + "┏" + "━"*78 + "┓")
print("┃" + " 💾 ARQUIVOS GERADOS".center(78) + "┃")
print("┣" + "━"*78 + "┫")
print(f"┃                                                                                ┃")
print(f"┃  1️⃣  DADOS LIMPOS:                                                             ┃")
print(f"┃     📂 Pasta: 02_Processados/                                                 ┃")
print(f"┃     📄 Arquivo: {arquivo_limpo.name[:60]:<60}┃")
print(f"┃                                                                                ┃")
print(f"┃  2️⃣  DICIONÁRIO DE CAMPOS:                                                     ┃")
print(f"┃     📂 Pasta: 03_Outputs/                                                     ┃")
print(f"┃     📄 Arquivo: {arquivo_dict.name[:60]:<60}┃")
print(f"┃                                                                                ┃")
print(f"┃  3️⃣  LOG DE PROCESSAMENTO:                                                     ┃")
print(f"┃     📂 Pasta: 04_Logs/                                                        ┃")
print(f"┃     📄 Arquivo: {arquivo_log.name[:60]:<60}┃")
print(f"┃                                                                                ┃")
print(f"┃  4️⃣  CÓDIGO PYTHON (REPRODUÇÃO):                                               ┃")
print(f"┃     📂 Pasta: 03_Outputs/                                                     ┃")
print(f"┃     📄 Arquivo: {arquivo_codigo.name[:60]:<60}┃")
print(f"┃                                                                                ┃")
print("┗" + "━"*78 + "┛")

print("\n" + "┏" + "━"*78 + "┓")
print("┃" + " 💡 PRÓXIMOS PASSOS".center(78) + "┃")
print("┣" + "━"*78 + "┫")
print("┃                                                                                ┃")
print("┃  ✅ Use a variável 'df_resultado' para continuar análises neste notebook      ┃")
print("┃  ✅ Importe o arquivo limpo em outros notebooks/scripts                       ┃")
print("┃  ✅ Use o código de reprodução para automatizar carregamentos futuros         ┃")
print("┃  ✅ Consulte o dicionário para entender os tipos de cada campo                ┃")
print("┃                                                                                ┃")
print("┗" + "━"*78 + "┛")

print("\n" + "┏" + "━"*78 + "┓")
print("┃" + " 🎯 LOCALIZAÇÃO DOS ARQUIVOS".center(78) + "┃")
print("┣" + "━"*78 + "┫")
print(f"┃                                                                                ┃")
print(f"┃  📂 Base do projeto:                                                          ┃")
print(f"┃     {str(fm.base_path):<74}┃")
print(f"┃                                                                                ┃")
print(f"┃  📂 Dados processados:                                                        ┃")
print(f"┃     {str(fm.pastas['processados']):<74}┃")
print(f"┃                                                                                ┃")
print(f"┃  📂 Outputs e relatórios:                                                     ┃")
print(f"┃     {str(fm.pastas['outputs']):<74}┃")
print(f"┃                                                                                ┃")
print("┗" + "━"*78 + "┛")

# Variável global para uso futuro
df_resultado = df_limpo.copy()

print("\n\n✅ PROCESSAMENTO 100% CONCLUÍDO!")
print("🚀 Abrindo pasta de outputs...")




╔==============================================================================╗
║                  📋 RELATÓRIO FINAL - PROCESSAMENTO CONCLUÍDO                 ║
╚==============================================================================╝

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                           📁 INFORMAÇÕES DO ARQUIVO                           ┃
┣━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┫
┃  Nome: 2025-2024-YSMM_VI_ACOMP.xlsx                                        ┃
┃  Sheet: 2025-2024-YSMMVIMONITOR                                            ┃
┃  Cabeçalho: Linha 4 (Excel) / Índice 3 (Python)                    ┃
┃  Dados: A partir da linha 5 (Excel)                                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                         📊 RESULTADO DO P

In [17]:
# ═══════════════════════════════════════════════════════════════════
# ABERTURA AUTOMÁTICA DA PASTA DESTINO
# ═══════════════════════════════════════════════════════════════════

# Abrir pasta de outputs automaticamente
fm.abrir_pasta('outputs')

print("\n💡 TIP: Se a pasta não abriu, o caminho está exibido no relatório acima!")


📂 Pasta aberta: C:\Users\fpsou\PycharmProjects\AIVI-RECALCULOBatentesLimites\03_Outputs

💡 TIP: Se a pasta não abriu, o caminho está exibido no relatório acima!


In [16]:
df_resultado

,Nome do set,Mês do exercício,Ano do documento do material,Centro,Cód Grupo de produto,Desc. Grupo de Produto,Nome,Expedição c/ Veí,Variação Interna,Variação Manual,...,Imposto (R$),Valor Exced. da,Valor da VI (R$),Valor da VI +,Perda ou So,Competência,Status de Homologação,Desc Status,Icone,Status
0,NaN,10.0,2025.0,5174.0,ETANOL_ADITIVADO,ETANOL ADITIVADO,BADEN Base de PresidPrudente,150837.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
1,NaN,10.0,2025.0,5174.0,GASOLINA_ADITIVADA,GASOLINA ADITIVADA,BADEN Base de PresidPrudente,222017.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
2,NaN,10.0,2025.0,5174.0,GASOLINA_COMPOSTO,GASOLINA,BADEN Base de PresidPrudente,1159915.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
3,NaN,10.0,2025.0,5174.0,GASOLINA_SIMPLES,GASOLINA,BADEN Base de PresidPrudente,1209443.0,-4659.5,0.0,...,0.0,-27535.3153,-27535.3153,-27535.3153,P,SUPER,A,Aguardando Envio para Estoquistas,@0A@,NaN
4,NaN,10.0,2025.0,5174.0,HIDRATADO_SIMPLES,HIDRATADO,BADEN Base de PresidPrudente,1703289.0,919.0,0.0,...,0.0,2703.9737,2703.9737,2703.9737,S,LM,A,Aguardando Envio para Estoquistas,@0A@,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27650,NaN,10.0,2025.0,5283.0,DIESEL_S10_ADITIVADO,DIESEL S10 ADITIVADO,AIPAF Área IndPool Passo Fundo,179677.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
27651,NaN,10.0,2025.0,5283.0,DIESEL_S10_COMPOSTO,DIESEL S10,AIPAF Área IndPool Passo Fundo,1812791.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
27652,NaN,10.0,2025.0,5283.0,DIESEL_S10_SIMPLES,DIESEL S10,AIPAF Área IndPool Passo Fundo,1694320.0,123.0,0.0,...,0.0,602.1588,602.1588,602.1588,S,LM,A,Aguardando Envio para Estoquistas,@0A@,NaN
27653,NaN,10.0,2025.0,5283.0,DIESEL_S500_ADITIVAD,DIESEL S500 ADITIVADO,AIPAF Área IndPool Passo Fundo,68901.0,0.0,0.0,...,0.0,0.0000,0.0000,0.0000,P,LT,A,Aguardando Envio para Estoquistas,@0A@,NaN
